In [85]:
# Multiple Sites (MS) - PE (100) / A (100%)

# checks if there are multiple tumor sites *THAT REQUIRE MOHS*
import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

system_context = """
You are reviewing dermatology notes to determine whether TWO OR MORE distinct lesions are actively planned for Mohs surgery in this clinical encounter.

Respond “Yes” ONLY if ALL of the following conditions are clearly met:

1. **TWO OR MORE BCC/SCC LESIONS**:
   - At least 2 **anatomically distinct lesions**, or clearly separate biopsy sites (e.g., cheek AND temple; Lesion A AND B).
   - Each lesion must be confirmed as **BCC** or **SCC** (ignore AK, melanoma, nevi, or "neoplasm of uncertain behavior").

2. **CURRENT MOHS PLANNING FOR EACH**:
   - Mohs must be explicitly stated in the current visit as the plan for BOTH sites:
     - Phrases like "Mohs scheduled," "Mohs recommended," "Phone Mohs completed," or "plan to do Mohs for both."
     - If a **shared Mohs plan** is described for multiple lesions (e.g., “Mohs same day”), count as YES — if no other treatment is listed.

3. **NO MIXED MANAGEMENT**:
   - If ANY of the lesions is being treated with excision, ED&C, topical treatments (like Efudex), or monitoring — respond “No.”
   - Exception: If Efudex is mentioned for field therapy **but Mohs is still planned and scheduled for specific lesions**, count as “Yes.”

4. **SAME ENCOUNTER, ACTIVE PLAN**:
   - All decisions must occur in this visit — not historical or vague plans.
   - Consent, scheduling, or coordination for Mohs must be clearly documented now.

Clarifications:
- “Mohs for lesions A and B” = Yes.
- “Plan to treat both sites with Mohs” = Yes, unless alternate treatments are listed.
- “Mohs for A, excision for B” = No.
- If biopsy confirmation isn’t explicitly restated but clearly inferred (e.g., Mohs coordination follows a named SCC), accept.

Be strict about treatment type, but flexible about phrasing when intent is clear for 2+ cancers.

Respond ONLY with: Yes or No
"""








def call_gpt(query):
    
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }

    payload = {
    "messages" : [
            {"role": "system", "content": system_context},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 800
    }
    
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        print("Response Content:", response.text)
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    #return response.json().choices[0].message.content
    return response.json()["choices"][0]["message"]["content"]

In [86]:
# Use full Excel sheet
import pandas as pd
pd.set_option('display.max_colwidth', 50)

Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
Cat.head()

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Recurrent Tumor (R),"Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)",Aggressive Tumor Pathology Type (AT),DFSP (D),Coordinated repair with ENT/oculoplastics (C),Additional C&F,Additional Excision,Multiple Sites,Business Days from Referral to First Appt,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,Y,N,N,N,N,N,N,N,32.0,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Y,Y,N,N,N,N,N,N,40.0,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,N,N,N,N,N,N,N,N,4.0,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,N,N,N,N,Y,N,N,N,35.0,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,N,N,N,N,N,Y,Y,N,35.0,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [87]:
# Set options here
sample_size = len(Cat) # Change number to increase or decrease number of patients read from file
#sample_size = 10
multiple_columns = False # Set to True to use all input text columns, False to use only PE column

# Array of indexes containing patients with notes that have problems (Some may be ok, these are from size so a location could exist but not size)
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = False # Set to True to skip bad patient notes, False to keep them in

In [88]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return "Yes" or "No" to the question
# May use only A column
import re

# Always split into 3 GPT calls, one for each column
def inputSplit(input_columns, que, index):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{que} \n\n {input_string}" # Create base prompt (question + input_columns)
    
    if(len(input_string) > 0):
        tmp_ans = [""] * len(input_columns)
        check_ans = [""] * len(input_columns)
        
        # Split into 3 GPT calls, one per column, and remove anything that isn't a "Yes" or "No" from the result
        for i in range(len(input_columns)):
            prompt = f"{que} \n\n {input_columns[i]}" # Recreate prompt with question + 1 column
            tmp_ans[i] = call_gpt(prompt) # Call GPT with prompt and save answer to array
            
            print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
            
            check_ans[i] = re.split(r'\s+|-', tmp_ans[i])  # Split by spaces or hyphens
            check_ans[i] = [re.sub(r'[^a-zA-Z]', '', j) for j in check_ans[i]]  # Remove non-alphabetic characters
            check_ans[i] = [j.lower() for j in check_ans[i]]  # Convert all to lowercase to handle variations

            check_ans[i] = [j for j in check_ans[i] if j in ['yes', 'no']] # Keep only "yes" or "no"
            
            print(f"check_ans[{i}] = {str(check_ans[i])}")
            
        # Check if 2 arrays are the same output, if so return one of them if it is not empty
        if(check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]):
            if check_ans[0]:
                return tmp_ans[0]
            elif check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]
        elif(check_ans[1] == check_ans[2]):
            if check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]

        # Count num of "yes" and "no"
        yes_count = [0] * len(input_columns)
        no_count = [0] * len(input_columns)
        
        for i in range(len(check_ans)):
            yes_count[i] = check_ans[i].count('yes')
            no_count[i] = check_ans[i].count('no')
            
        print(f"Yes counts: {yes_count}")
        print(f"No counts: {no_count}")
        
        # If no matches, return string that has the most "yes" or "no"
        if any(count > 0 for count in yes_count) or any(count > 0 for count in no_count): # Check count strings for any occurences or "yes" or "no"
            max_yes_no = [0] * len(input_columns)
            max_answer = 0
            max_i = -1
            
            # Check if string has more "yes" or "no"
            for i in range(len(input_columns)):
                max_yes_no[i] = max(yes_count[i], no_count[i])
                
            # Look for highest overall "yes" or "no" then return that string
            for i in range(len(input_columns)):
                if max_answer < max_yes_no[i]:
                    max_answer = max_yes_no[i]
                    max_i = i
            return tmp_ans[max_i]
        
        # All non "yes" or "no" answers, return empty string
        return ""
    else:
        # Somehow input_string is empty
        return "input_string empty"
    

true_results = [0 for i in range(sample_size)]
true_results_final = [0 for i in range(sample_size)]

output = [0 for i in range(sample_size)]
output_final = [0 for i in range(sample_size)]

for i in range(0, min(sample_size, len(Cat))):
    # If skip_patients is True and current index is in skip_patients_array, move to next index
    if skip_patients and i in skip_patients_array:
        continue
        
    true_results[i] = Cat.at[i, 'Multiple Sites']
    que = "Are two or more biopsy-confirmed lesions in this note planned for surgical treatment in this visit?"
    ans = ""
    
    if multiple_columns:
        input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
        ans = inputSplit(input_columns, que, i) # Result from GPT call
    elif not multiple_columns:
        input_columns = Cat.at[i, 'Addendum (A)']
        #input_columns = Cat.at[i, 'Assessment & Plan (A&P)']
        
        prompt = f"{que} \n\n {input_columns}"
        ans = call_gpt(prompt)
    
    #print(f"Result[{i}] = {ans}")
    output[i] = ans
    print(i)
    
    print("-" *50)
    

0
--------------------------------------------------
1
--------------------------------------------------
2
--------------------------------------------------
3
--------------------------------------------------
4
--------------------------------------------------
5
--------------------------------------------------
6
--------------------------------------------------
7
--------------------------------------------------
8
--------------------------------------------------
9
--------------------------------------------------
10
--------------------------------------------------
11
--------------------------------------------------
12
--------------------------------------------------
13
--------------------------------------------------
14
--------------------------------------------------
15
--------------------------------------------------
16
--------------------------------------------------
17
--------------------------------------------------
18
-----------------------------------

In [89]:
import re

# Go through every GPT response and turn it into "Y" or "N" to compare
for i in range(0, min(sample_size, len(Cat))):
    if skip_patients and i in skip_patients_array:
        continue
        
    yn = output[i]
    #print(f"i={i} Answer: {yn}")
    yn_real = str(true_results[i])
    
    yn = re.split(r'\s+|-', yn)  # Split by spaces or hyphens
    yn = [re.sub(r'[^a-zA-Z]', '', j) for j in yn]  # Remove non-alphabetic characters
    yn = [j.lower() for j in yn]  # Convert all to lowercase to handle variations
    
    yn = [j for j in yn if j in ['yes', 'no']] # Keep only "yes" or "no"
    
    if len(yn) == 0:
        output_final[i] = "error_2"  # No valid response detected, bad prediction
    
    yes_count = yn.count('yes')
    no_count = yn.count('no')
    
    if yes_count > no_count:
        output_final[i] = "Y"
    elif no_count > yes_count:
        output_final[i] = "N"
        
    # Check if the given input is valid
    if yn_real.strip() and yn_real.lower() != 'nan':
        if 'y' in yn_real.lower():
            true_results_final[i] = "Y" # Some are input like N/Y or Y/N/N so look for Y and if exists compare with that
        else:
            true_results_final[i] = yn_real.strip()
    else:
        true_results_final[i] = "error_1" # No valid input provided in true_results, bad note
    
    print(f"Result[{i}]: ChatGPT Prediction = {output_final[i]} Answer Given = {true_results_final[i]}")

Result[0]: ChatGPT Prediction = N Answer Given = N
Result[1]: ChatGPT Prediction = N Answer Given = N
Result[2]: ChatGPT Prediction = N Answer Given = N
Result[3]: ChatGPT Prediction = N Answer Given = N
Result[4]: ChatGPT Prediction = N Answer Given = N
Result[5]: ChatGPT Prediction = N Answer Given = N
Result[6]: ChatGPT Prediction = N Answer Given = N
Result[7]: ChatGPT Prediction = N Answer Given = N
Result[8]: ChatGPT Prediction = N Answer Given = N
Result[9]: ChatGPT Prediction = N Answer Given = N
Result[10]: ChatGPT Prediction = N Answer Given = N
Result[11]: ChatGPT Prediction = N Answer Given = N
Result[12]: ChatGPT Prediction = N Answer Given = N
Result[13]: ChatGPT Prediction = N Answer Given = N
Result[14]: ChatGPT Prediction = N Answer Given = N
Result[15]: ChatGPT Prediction = N Answer Given = N
Result[16]: ChatGPT Prediction = N Answer Given = N
Result[17]: ChatGPT Prediction = N Answer Given = Y
Result[18]: ChatGPT Prediction = N Answer Given = N
Result[19]: ChatGPT Pr

In [90]:


# Configure pandas to show full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

for i in range(sample_size):
    if true_results[i] == "N" and output[i] == "Yes":
        print(f"\n==== False Pos at index {i} ====")
        # Print full text of both columns
        #print("[A&P]:", Cat.at[i, 'Assessment & Plan (A&P)'])
        print("[Addendum]:", Cat.at[i, 'Addendum (A)'])
        print("=" * 50)


==== False Pos at index 68 ====
[Addendum]: Addendum 9/1/2023
 
The pathology results were relayed via phone call (see below). The patient is healing well without redness or discharge. 
 
Discussed rash consistent with tinea corporis.  Explained treatment option could include prolonged course of oral azole antifungal.  Patient is not interested in treatment at this time.  
 
Will plan for no further treatment on the left medial cheek due to goal of punch excision with negative margins.
 
Will plan for Mohs of the lesion on the right temple.
 
Patient notified of biopsy result, advised Mohs surgery due to location where tissue conservation is critical.  Discussed risks (including but not limited to infection, bleeding, scar formation), benefits and alternatives as well as repair options that are often uncertain until the lesion has been excised, discussed that the surgical defect may be larger than the clinically evident lesion.  Reviewed the Mohs surgery process and advised to reserve

In [91]:


# Configure pandas to show full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

for i in range(sample_size):
    if true_results[i] == "Y" and output[i] == "No":
        print(f"\n==== False Neg at index {i} ====")
        # Print full text of both columns
        #print("[A&P]:", Cat.at[i, 'Assessment & Plan (A&P)'])
        print("[Addendum]:", Cat.at[i, 'Addendum (A)'])
        print("=" * 50)


==== False Neg at index 17 ====
[Addendum]: Addendum 7/20/2023
The pathology results were relayed via phone call (see below). The patient is healing well without redness or discharge. 
 
Diagnosis	 	 
 	Skin, vertex scalp, shave:
            Basal cell carcinoma
 
I have personally reviewed this case and edited the report as necessary.
 
 
At this time, patient elects for Mohs surgery. Advised Mohs surgery due to location where tissue conservation is critical.  Discussed risks (including but not limited to infection, bleeding, scar formation) and benefits as well as repair options that are often uncertain until the lesion has been excised, discussed that the surgical defect may be larger than the clinically evident lesion.  Reviewed the Mohs surgery process and advised to reserve the entire day for the procedure.  Advised to continue all medications. Preoperative antibiotics are not indicated.  Phone Mohs completed and submitted for scheduling.
 
There are two lesions that may require

In [92]:
total_size = 0
correct_predictions = 0
no_response_given = 0
no_alphabetical_result_returned = 0
bad_cases_combined = 0
nrg_not_narr = 0
narr_not_nrg = 0
# nrg = no valid response given in (L) column
# narr = no alphabetical result returned from ChatGPT output

for i in range(0, min((sample_size), len(Cat))):
    if(output_final[i] == "error_2" and true_results_final[i] == "error_1"):
        bad_cases_combined += 1
        
    if(output_final[i] == "error_2"):
        no_alphabetical_result_returned += 1
        if(true_results_final[i] != "error_1"):
            narr_not_nrg += 1
            
    if(true_results_final[i] == "error_1"):
        no_response_given += 1
        if(output_final[i] != "error_2"):
            nrg_not_narr += 1
    
    if(output_final[i] == true_results_final[i]):
        correct_predictions += 1
        
    total_size += 1
    
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))

# Accuracy when removing cases where the Location is not given (best to look at)
no_response_given_accuracy = correct_predictions / (total_size - no_response_given)
print()
print("Number of Notes without a Given Multiple Sites = " + str(no_response_given))
print("Accuracy = " +"{:.2f}%".format(no_response_given_accuracy * 100))

# Accuracy when removing cases where GPT didn't return a valid alphabetical answer (not the best to look at, could be slightly lower)
no_alphabetical_result_returned_accuracy = correct_predictions / (total_size - no_alphabetical_result_returned)
print()
print("Number of PE/A&P/A Notes without a valid GPT Given Multiple Sites = " + str(no_alphabetical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_alphabetical_result_returned_accuracy * 100))

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (narr_not_nrg + nrg_not_narr + bad_cases_combined))
print()
print("Number of Combined Notes without a Given Multiple Sites = " + str(narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))     

# of Correct Predictions = 274
Sample Size = 294
Accuracy = 93.20%

Number of Notes without a Given Multiple Sites = 3
Accuracy = 94.16%

Number of PE/A&P/A Notes without a valid GPT Given Multiple Sites = 0
Accuracy = 93.20%

Number of Combined Notes without a Given Multiple Sites = 3
Accuracy = 94.16%


In [93]:
n_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i].strip() != true_results_final[i].strip() and true_results_final[i] != "error_1" and output[i] != "Yes":
        n_count += 1
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print("-"*40) # Spacer
print(f"n_count = {n_count}")

i = 17: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 31: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 38: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 126: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 131: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 187: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 197: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 216: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 228: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 239: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 253: ChatGPT Prediction = N AT Given = Y
----------------------------------------
i = 254: ChatGPT Prediction = N AT Given = Y
------------

In [94]:
y_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and output[i] != "No":
        y_count += 1
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print("-"*40) # Spacer
print(f"y_count = {y_count}")

i = 68: ChatGPT Prediction = Y AT Given = N
----------------------------------------
i = 89: ChatGPT Prediction = Y AT Given = N
----------------------------------------
i = 96: ChatGPT Prediction = Y AT Given = N
----------------------------------------
i = 143: ChatGPT Prediction = Y AT Given = N
----------------------------------------
i = 153: ChatGPT Prediction = Y AT Given = N
----------------------------------------
y_count = 5


In [95]:
count = 0
for i in range(0, min((sample_size), len(Cat))):
    if true_results_final[i] == "N":
        count += 1
print(f"no_count = {count}")
# 24 and 113 are bad notes for comparing, NA on every ground truth column

no_count = 229


In [96]:
def confusionMatrix():
    TP = TN = FP = FN = 0
    for i in range(0, min((sample_size), len(Cat))):
        if output_final[i] == true_results_final[i]: # True+/True-
            if output_final[i] == "Y": # True Positive
                TP += 1
            elif output_final[i] == "N": # True Negative
                TN += 1
        elif(output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and true_results_final[i] != "error_2"): # False+/False-
            if output_final[i] == "Y": # False Positive
                FP += 1
            elif output_final[i] == "N": # False Negative
                FN += 1
    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN}

def confusionMetrics(confusion_matrix):
    TP = confusion_matrix["TP"]
    TN = confusion_matrix["TN"]
    FP = confusion_matrix["FP"]
    FN = confusion_matrix["FN"]
    
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / (TP + TN + FP + FN) if total else 0 # How often did model predict correctly
    precision = TP / (TP + FP) if (TP + FP) else 0 # When model predicted Y, how often was it correct
    recall = TP / (TP + FN) if (TP + FN) else 0 # How many of the true Y cases did model correctly identify
    F1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0 # Balance between Precision and Recall
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "F1": F1}

            
total_size = 0
correct_predictions = 0
no_response_given = 0
no_alphabetical_result_returned = 0
bad_cases_combined = 0
nrg_not_narr = 0
narr_not_nrg = 0
# nrg = no valid response given in (L) column
# narr = no alphabetical result returned from ChatGPT output

for i in range(0, min((sample_size), len(Cat))):
    if(output_final[i] == "error_2" and true_results_final[i] == "error_1"):
        bad_cases_combined += 1
        
    if output_final[i] == "error_2":
        no_alphabetical_result_returned += 1
        if true_results_final[i] != "error_1":
            narr_not_nrg += 1
            
    if true_results_final[i] == "error_1":
        no_response_given += 1
        if output_final[i] != "error_2":
            nrg_not_narr += 1
    
    if output_final[i] == true_results_final[i]:
        correct_predictions += 1
        
    total_size += 1
    
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))
print()

# Accuracy when removing cases where the Location is not given (best to look at)
no_response_given_accuracy = correct_predictions / (total_size - no_response_given)
print("Number of Notes without a Given MS = " + str(no_response_given))
print("Accuracy = " +"{:.2f}%".format(no_response_given_accuracy * 100))
print()

# Accuracy when removing cases where GPT didn't return a valid alphabetical answer (not the best to look at, could be slightly lower)
no_alphabetical_result_returned_accuracy = correct_predictions / (total_size - no_alphabetical_result_returned)
print("Number of PE/A&P/A Notes without a valid GPT Given MS = " + str(no_alphabetical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_alphabetical_result_returned_accuracy * 100))
print()

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Number of Combined Notes without a Given MS = " + str(narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))
print()

confusion_matrix = confusionMatrix()
confusion_metrics = confusionMetrics(confusion_matrix)

print("Confusion Matrix:")
for name, value in confusion_matrix.items():
    print(f"{name}: {value}")

print("\nConfusion Matrix Metrics:")
for name, value in confusion_metrics.items():
    print(f"{name}: {(value * 100):.2f}")

# of Correct Predictions = 274
Sample Size = 294
Accuracy = 93.20%

Number of Notes without a Given MS = 3
Accuracy = 94.16%

Number of PE/A&P/A Notes without a valid GPT Given MS = 0
Accuracy = 93.20%

Number of Combined Notes without a Given MS = 3
Accuracy = 94.16%

Confusion Matrix:
TP: 50
TN: 224
FP: 5
FN: 12

Confusion Matrix Metrics:
accuracy: 94.16
precision: 90.91
recall: 80.65
F1: 85.47
